In [1]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_openai import ChatOpenAI
import json

# Load environment variables from the .env file
load_dotenv()

# Database file path
database_file_path = "./db/test.db"

# Create the SQLAlchemy engine
engine = create_engine(f'sqlite:///{database_file_path}')

@tool
def calculate_price(product_list: list) -> float:
    """
    Calculates the total price of a list of products.

    Args:
        product_list (list): A list of dictionaries representing products.
        
    Returns:
        float: The total price of the products.
    """
    total_price = sum(product['price'] for product in product_list)
    return total_price

@tool
def get_product_price(product_name: str) -> str:
    """
    Retrieves the price of a specific product.

    Args:
        product_name (str): The name of the product.
        
    Returns:
        str: The price of the product in JSON format.
    """
    try:
        query = text("""
            SELECT name, price
            FROM Product
            WHERE name LIKE :product_name;
        """)

        with engine.connect() as connection:
            result = pd.read_sql_query(query, connection, params={"product_name": f"%{product_name}%"})
        
        if not result.empty:
            return json.dumps(result.to_dict('records')[0])
        else:
            return json.dumps({"product_name": product_name, "price": None})
    except Exception as e:
        return json.dumps({"error": str(e)})

@tool
def get_products_available() -> str:
    """
    Retrieves the list of products that are available.

    Returns:
        str: A list of available products in JSON format.
    """
    try:
        query = text("""
            SELECT name, price
            FROM Product
            WHERE available = 1;
        """)

        with engine.connect() as connection:
            result = pd.read_sql_query(query, connection)
        
        if not result.empty:
            return json.dumps(result.to_dict('records'))
        else:
            return json.dumps({"available_products": []})
    except Exception as e:
        return json.dumps({"error": str(e)})

# Define the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# Define the tools
tools_sql = [get_product_price, get_products_available, calculate_price]

# Get the API key from environment variables
api_key = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=api_key)

# Create the agent
agent = create_tool_calling_agent(llm, tools_sql, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools_sql, verbose=True)

# Example usage
# agent_executor.invoke({"input": "What is the total cost for a portable charger and a keyboard?"})


In [2]:
# Prueba la ejecución del agente
# agent_executor.invoke({"input": "Quiero comprar Un teclado un una camara cuanto me costaria"})

# Prueba de disponibilidad de productos
# agent_executor.invoke({"input": "Que productos tienes disponibles?"})
agent_executor.invoke({"input": "Quiero comprar un Cargador portátil y un teclado, cual es el total?"})



> Entering new AgentExecutor chain...

Invoking: `get_products_available` with `{}`


[{"name": "Teclado", "price": 75.25}, {"name": "Laptop", "price": 1000.0}, {"name": "Smartphone", "price": 800.0}, {"name": "Tablet", "price": 300.0}, {"name": "Impresora", "price": 200.0}, {"name": "Mano", "price": 150.0}, {"name": "C\u00e1mara", "price": 400.0}, {"name": "Televisor", "price": 700.0}, {"name": "Router", "price": 80.0}, {"name": "Reproductor Blu-ray", "price": 180.0}, {"name": "Teclado inal\u00e1mbrico", "price": 60.0}, {"name": "Mouse inal\u00e1mbrico", "price": 80.0}, {"name": "Webcam", "price": 70.0}, {"name": "Tarjeta de video", "price": 250.0}, {"name": "Memoria RAM", "price": 120.0}, {"name": "Disco duro externo", "price": 150.0}, {"name": "Tarjeta madre", "price": 350.0}, {"name": "Procesador", "price": 300.0}, {"name": "Gabinete para PC", "price": 80.0}, {"name": "Fuente de poder", "price": 100.0}, {"name": "Router inal\u00e1mbrico", "price": 50.0}, {"name": "Adaptador Wi-Fi

{'input': 'Quiero comprar un Cargador portátil y un teclado, cual es el total?',
 'output': 'El total de comprar un Cargador portátil y un teclado es de $115.25.'}